# Deliverable 2
This is the notebook that has the second deliverable.

# 0. Import modules
Feel free to use the virtual environment (amonavis) included in the GitHub folder.

In [29]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt 
import networkx

# For semantic similarity
from urllib.parse import unquote
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Python functions in .py file to read data
import data_readers

# 1. Data reading
The following code reads the data and saves them in the appropriate variables.

<br><br><br>
**finished_paths**

The datafile includes the hashedIpAddress, timestamp, durationInSec, path, and rating.

We also add columns with the first article (soruce), last article (target), path length (#articles), and a readable date in Timestamp format.

<br><br>
**articles**

Dataframe with the name of all articles in the dataset.


<br><br><br>


<div style="border: 2px solid white; padding: 10px;">
    <font color="red">please briefly describe the other files. As I only worked with finished_paths and articles, I guess it's better if someone who used them does that. Also add a sample of how each data looks like. </font>
</div>

In [26]:
# The links and edges
wikispeedia = data_readers.read_wikispeedia_graph()

# The finished paths
finished_paths = data_readers.read_finished_paths()

# The unfinished paths
unfinished_paths = data_readers.read_unfinished_paths()

# List of all articles
articles = data_readers.read_articles()

# The shortest path matrix
# This one is the slowest to read by far, probably due to the weird parsing that has to be done!
shortest_path_df = data_readers.read_shortest_path_df()

# Searching for the string of a given article. It has to be formatted like the article name
# Which shouldn't be a problem, as we'll probably usually retrieve them internally
obi_wan_text = data_readers.plaintext_article_finder('Obi-Wan_Kenobi')

In [36]:
wikispeedia.head() # This obviously doesn't work but I don't know how to fix it xd

AttributeError: 'DiGraph' object has no attribute 'head'

In [30]:
finished_paths.sample(5)

,hashedIpAddress,timestamp,durationInSec,path,rating,first_article,last_article,path_length,date
29743,731282ad7471ca17,1257536736,28,Ozone;Sun;Planet;Venus,NaN,Ozone,Venus,4,2009-11-06 19:45:36
45796,5f570722327e17e9,1249599855,125,Ramesses_II;Israel;United_Kingdom;River_Thames,NaN,Ramesses_II,River_Thames,4,2009-08-06 23:04:15
12885,5975c03e7ce87ba5,1351323019,142,Asteroid;Mars;Roman_mythology;Mythology;Norse_...,3.0,Asteroid,Viking,9,2012-10-27 07:30:19
20788,7f6eb13a10c0d8a5,1373348817,151,Pyramid;Mexico;Agriculture;Soybean;<;Rice;Seed...,2.0,Pyramid,Bean,8,2013-07-09 05:46:57
45242,276f5f9603acc0b9,1359682562,165,Plymouth;England;<;Ghana;Africa;Sierra_Leone,NaN,Plymouth,Sierra_Leone,6,2013-02-01 01:36:02


In [33]:
unfinished_paths.sample(5)

,hashedIpAddress,timestamp,durationInSec,path,target,type
7713,28ed64876d7d2fcb,1330484522,259,Tunis;Capital;City;United_States;Cinema_of_the...,King_Kong_%282005_film%29,restart
17741,522c265d73d7ace6,1365024347,1824,Albania;Roman_Empire,ALCO_FA,timeout
3822,100ae36c715fdfc9,1306551464,183,Space_Shuttle_program;International_Space_Stat...,Great_Wall_of_China,restart
23307,3a4945025802b208,1384358526,69,Drought;Agriculture;Food;English_language;Unit...,Krazy_Kat,restart
12161,6b53e12119b3efb0,1347740880,64,Extrasolar_planet;Solar_System,Three_Laws_of_Robotics,restart


In [31]:
articles.sample(5)

,articles
1605,Funchal
61,90377_Sedna
3852,Spring_%28season%29
1456,Extraterrestrial_life
416,Avro_Lancaster


In [38]:
shortest_path_df.sample(5)

,"(%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,)","(%C3%85land,)","(%C3%89douard_Manet,)","(%C3%89ire,)","(%C3%93engus_I_of_the_Picts,)","(%E2%82%AC2_commemorative_coins,)","(10th_century,)","(11th_century,)","(12th_century,)","(13th_century,)",...,"(Ziad_Jarrah,)","(Zimbabwe,)","(Zinc,)","(Zinc_chloride,)","(Zion_National_Park,)","(Zionism,)","(Zirconium,)","(Zoroaster,)","(Zuid-Gelders,)","(Zulu,)"
"(X-Men__The_Last_Stand,)",-1,-1,-1,-1,-1,-1,3,3,3,2,...,3,3,2,3,3,3,3,3,4,3
"(Water_polo,)",-1,-1,-1,-1,-1,-1,3,3,3,3,...,3,3,3,4,4,3,4,3,4,3
"(Greek_War_of_Independence,)",-1,-1,-1,-1,-1,-1,2,2,2,2,...,4,3,3,4,4,2,4,3,4,2
"(Tom_and_Jerry_%28MGM%29,)",-1,-1,-1,-1,-1,-1,3,3,3,3,...,3,3,3,4,3,3,4,3,3,3
"(Mao_Zedong,)",-1,-1,-1,-1,-1,-1,2,2,2,2,...,3,3,3,4,3,3,4,3,3,3


In [39]:
obi_wan_text

'   #copyright\n\nObi-Wan Kenobi\n\n2007 Schools Wikipedia Selection. Related subjects: Films\n\n   Star Wars character\n   Obi-Wan "Ben" Kenobi\n     __________________________________________________________________\n\n   Position Jedi Master, Jedi Council Member, General of the Republic\n   Homeworld Unknown\n   Species Human\n   Gender Male\n   Height 1.75 m\n   Affiliation Jedi Order, Galactic Republic, Rebel Alliance\n   Portrayed by Ewan McGregor (Episodes I-III)\n   Sir Alec Guinness (Episodes IV-VI)\n\n   Obi-Wan "Ben" Kenobi ( 57 – 0 BBY) is a fictional character in the Star\n   Wars universe. He is one of the most prominent characters in the Star\n   Wars saga; along with Anakin Skywalker, R2-D2, and C-3PO, he is one of\n   the few major characters to have appearances (in some form or another)\n   in each of the six Star Wars films. He was portrayed in Episodes I-III\n   by Ewan McGregor, and in Episodes IV-VI by Sir Alec Guinness. He is\n   voiced by Bernard Behrens in the 

# 2. Study of Unique Paths
Here we study the unique source and target pairs. We will use the dataframes to compare the performance between humans and machines, as well as to know what paths to make machines complete.

**article_combinations**

This dataframe contains information on all the combination of source and target articles in the finished games (paths). It includes how many times it has been played, and the mean and std of the path length, duration of the game, and rating.

**unique_targets** and **unique_sources**

These dataframes include all the sources and targets that appears in the finished games

<br><br>
Note that we don't change to ASCII the name of the articles yet. We will do it at a later step if we need to.
<br><br>


<div style="border: 2px solid white; padding: 10px;">
    <font color="red">feel free to move this wherever works better. Maybe not the best thing to have at the beginning of the notebook.</font>
</div>

In [23]:
# How many each pair of articles has been visited
article_combinations_count = finished_paths.groupby(['first_article', 'last_article']).size().reset_index(name='count')

# The mean and std of the path length for each pair of articles
article_combinations_stats = finished_paths.groupby(['first_article', 'last_article'])['path_length'].agg(['mean', 'std']).reset_index()
article_combinations_stats['std'] = article_combinations_stats['std'].fillna(0)
article_combinations_stats.rename(columns={'mean': 'mean_length', 'std': 'std_length'}, inplace=True)

# The mean and std of the rating for each pair of articles. 
    # Note that mean and std may be nan if there are nan ratings. We purposely leave them as nan, as we don't want to fill them with 0s or 1s.
    # Depending on the application, we could change this in the future if neeeded.
rating_combinations_stats_rating = finished_paths.groupby(['first_article', 'last_article'])['rating'].agg(['mean', 'std']).reset_index()
#rating_combinations_stats_rating['std'] = rating_combinations_stats_rating['std'].fillna(0)
mask = rating_combinations_stats_rating['mean'].notnull()
rating_combinations_stats_rating.loc[mask, 'std'] = rating_combinations_stats_rating.loc[mask, 'std'].fillna(0)
rating_combinations_stats_rating.rename(columns={'mean': 'mean_rating', 'std': 'std_rating'}, inplace=True)

# The mean and std of the time for each pair of articles.
rating_combinations_stats_time = finished_paths.groupby(['first_article', 'last_article'])['durationInSec'].agg(['mean', 'std']).reset_index()
rating_combinations_stats_time['std'] = rating_combinations_stats_time['std'].fillna(0)
rating_combinations_stats_time.rename(columns={'mean': 'mean_durationInSec', 'std': 'std_durationInSec'}, inplace=True)

# Merging all the dataframes
article_combinations = pd.merge(article_combinations_count, article_combinations_stats, on=['first_article', 'last_article'])
article_combinations = pd.merge(article_combinations, rating_combinations_stats_rating, on=['first_article', 'last_article'])
article_combinations = pd.merge(article_combinations, rating_combinations_stats_time, on=['first_article', 'last_article'])

# The number of unique sources and targets
unique_sources = finished_paths['first_article'].value_counts().reset_index()
unique_targets = finished_paths['last_article'].value_counts().reset_index()

In [25]:
article_combinations.sample(5)

,first_article,last_article,count,mean_length,std_length,mean_rating,std_rating,mean_durationInSec,std_durationInSec
17915,Music_of_Ireland,Extrasolar_planet,1,6.00,0.0,2.0,0.0,28.00,0.000000
23580,Sodium,Angola,1,5.00,0.0,2.0,0.0,51.00,0.000000
12927,Hurricane_Ivan,Monaco,1,6.00,0.0,NaN,NaN,49.00,0.000000
15738,Liverpool,Union_Station_%28San_Diego%29,1,7.00,0.0,3.0,0.0,454.00,0.000000
3382,Bay_of_Bengal,Athens,4,5.75,0.5,4.0,0.0,96.25,62.419949


In [26]:
unique_sources.sample(5)

,first_article,count
1543,Dune,11
87,Barack_Obama,34
1856,Mysticism,10
3444,Stickleback,4
3847,Maghreb,3


In [28]:
unique_targets.sample(5)

,last_article,count
535,West_Virginia,22
3243,ASCII,4
2417,Nahuatl_language,6
2487,Damascus_steel,6
598,Newspaper,21


# 3. Semantic Similarity

An important part of the project is to study how humans and machines move from article to article. Semantic similarity compares two strings based on a trained model and assigns a value according to how are they correlated (the higher, the more related). For example, 'king' and 'queen' will have a higher semantic similarity than say, 'king' and 'chemistry' (will prove this here).

### Remove the underscore and decode the url

First we define a function that corrects the strings to have a readable format. For example, '%C3%89douard_Manet' is transformed to 'Édouard Manet'.

We will create a function to decode a word, and we will be able to use it in series and dataframes using apply().

In [10]:
def decode_word(word):
    word = word.replace('_', ' ')
    return unquote(word)

In [15]:
articles['articles'].head(5)

0    %C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1                          %C3%85land
2                  %C3%89douard_Manet
3                           %C3%89ire
4          %C3%93engus_I_of_the_Picts
Name: articles, dtype: object

In [14]:
articles['articles'].apply(decode_word).head(5)

0        Áedán mac Gabráin
1                    Åland
2            Édouard Manet
3                     Éire
4    Óengus I of the Picts
Name: articles, dtype: object

### Semantic Distance Model
We create a function that returns the semantic similarity between two words you provide.

In [2]:
# We define the model outside the function (make sure to run this before using the function)
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
# Function to get embeddings (just because we will use it in semantic similarity function)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Semantic similarity function
def semantic_similarity(word1, word2):
    embedding1 = get_embedding(word1)
    embedding2 = get_embedding(word2)
    return cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]

In [5]:
semantic_similarity('king', 'queen')

0.9388244

In [16]:
semantic_similarity('king', 'chemistry')

0.655022

### Semantic Distance Matrix
Provided a series, it creates a df where indices and column names are the strings of the series, and fills the matrix with the semantic similarity between all words in the provided series.

In [22]:
def semantic_similarity_matrix(titles):
    df = pd.DataFrame(index=titles, columns=titles)
    for i in range(len(titles)):
        for j in range(i+1, len(titles)):
            embedding1 = get_embedding(titles[i])
            embedding2 = get_embedding(titles[j])
            similarity = cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]
            df.iloc[i, j] = similarity
            df.iloc[j, i] = similarity  # Copy value to lower triangle
            np.fill_diagonal(df.values, 1)
    return df

In [23]:
semantic_similarity_matrix(pd.Series(['king', 'queen', 'chemistry', 'biology']))

,king,queen,chemistry,biology
king,1,0.938824,0.655022,0.734002
queen,0.938824,1,0.633365,0.745616
chemistry,0.655022,0.633365,1,0.846376
biology,0.734002,0.745616,0.846376,1
